In [1]:
import torch
USE_CUDA = torch.cuda.is_available()    # gpu 사용가능여부 >>> T/F
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')   # USE_CUDA가 True면 GPU 사용 아니면 cpu사용

In [2]:
EPOCHS = 5
BATCH_SIZE = 64

In [5]:
from torchvision import transforms, datasets

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('Data',   # 저장경로
                   train=True,
                   download=True,
                   transform=transforms.Compose([    
                       transforms.RandomHorizontalFlip(),   # random 가로방향으로 방향전환
                       transforms.ToTensor(),               # tensor로 변형
                       transforms.Normalize((0.1307,), (0.3081,))   # 평균, 표준편차를 지정 
                       # 0~255   color이미지는 3채널,   흑백이미지는 1채널
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('Data',
                   train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))  
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)



100%|██████████| 9912422/9912422 [00:00<00:00, 38226672.47it/s]


Extracting Data\MNIST\raw\train-images-idx3-ubyte.gz to Data\MNIST\raw


100%|██████████| 28881/28881 [00:00<00:00, 28952125.67it/s]


Extracting Data\MNIST\raw\train-labels-idx1-ubyte.gz to Data\MNIST\raw




  0%|          | 0/1648877 [00:00<?, ?it/s]

100%|██████████| 1648877/1648877 [00:00<00:00, 16934443.86it/s]


Extracting Data\MNIST\raw\t10k-images-idx3-ubyte.gz to Data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting Data\MNIST\raw\t10k-labels-idx1-ubyte.gz to Data\MNIST\raw



In [6]:
import torch.nn as nn
import torch.nn.functional as F

# 신경망모델 keras에서는 sequential, torch에서는 module
class Net(nn.Module):
    def __init__(self, dropout_p = 0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout_p = dropout_p

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training, p = self.dropout_p)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training= self.training, p = self.dropout_p)
        x = self.fc3(x)
        return x

        

In [7]:
import torch.optim as optim

# 과적합과 정규화
model = Net(dropout_p = 0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [10]:
def train(model, train_loader, optimizer):
    model.train()   # 학습모드: 순전파+역전파  
    for _, (data, target) in enumerate(train_loader):
        # 인덱스, 데이터
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()   # 경사 초기화
        output = model(data)    # 모델 입력
        loss = F.cross_entropy(output, target)
        loss.backward()         # 역전파계산
        optimizer.step()        # 반복

def evaluate(model, test_loader):
    model.eval()   # 추론모드: 순전파
    test_loss = 0
    correct = 0
    with torch.no_grad():   # no_grad 미분하지 않겠다
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)        
            output = model(data)

            # cross entropy
            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            # softmax 연산 (output에서 최대값 선택)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy
    

In [11]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print(f'[{epoch}] Test loss: {test_loss: .4f}, Accuracy: {test_accuracy: .2f}%')

[1] Test loss:  0.5371, Accuracy:  82.59%
[2] Test loss:  0.4160, Accuracy:  86.64%
[3] Test loss:  0.3401, Accuracy:  89.21%
[4] Test loss:  0.2914, Accuracy:  90.85%
[5] Test loss:  0.2459, Accuracy:  92.43%


In [ ]:
a
